In [2]:
#import dependencies
import pandas as pd

In [12]:
#open dataset
schools_csv = "Resources/schools_complete.csv"
students_csv = "Resources/students_complete.csv"

schools_df = pd.read_csv(schools_csv, index_col="School ID")
students_df = pd.read_csv(students_csv, index_col="Student ID")

In [13]:
#Check dataset consistency and stats
print(schools_df.describe())
print(schools_df.dtypes)
print(schools_df.count())
print(schools_df.head())

              size        budget
count    15.000000  1.500000e+01
mean   2611.333333  1.643295e+06
std    1420.915282  9.347763e+05
min     427.000000  2.480870e+05
25%    1698.000000  1.046265e+06
50%    2283.000000  1.319574e+06
75%    3474.000000  2.228999e+06
max    4976.000000  3.124928e+06
school_name    object
type           object
size            int64
budget          int64
dtype: object
school_name    15
type           15
size           15
budget         15
dtype: int64
                     school_name      type  size   budget
School ID                                                
0              Huang High School  District  2917  1910635
1           Figueroa High School  District  2949  1884411
2            Shelton High School   Charter  1761  1056600
3          Hernandez High School  District  4635  3022020
4            Griffin High School   Charter  1468   917500


In [14]:
#Check dataset consistency and stats
print(students_df.describe())
print(students_df.dtypes)
print(students_df.count())
print(students_df.head())

       reading_score    math_score
count    39170.00000  39170.000000
mean        81.87784     78.985371
std         10.23958     12.309968
min         63.00000     55.000000
25%         73.00000     69.000000
50%         82.00000     79.000000
75%         91.00000     89.000000
max         99.00000     99.000000
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64
                 student_name gender grade        school_name  reading_score  \
Student ID                                                                     
0                Paul Bradley      M   9th  Huang High School             66   
1                Victor Smith      M  12th  Huang High School             94   
2             Kevin Rodriguez      M  12th  Huang High S

In [15]:
#Set pass mark
maths_pass = 65
reading_pass = 65

Primary analysis
================

District Summary
------------------------

In [26]:
#Total schools
total_schools = schools_df.school_name.count()

#Total students
total_students = students_df.school_name.count()

#Total budget
total_budget = schools_df.budget.sum()

#Average maths score
average_maths = students_df.math_score.mean()

#Average reading score
average_reading = students_df.reading_score.mean()

#% passing maths
pc_pass_maths = students_df[students_df.math_score > maths_pass].count()["math_score"]*100/total_students

#% passing reading
pc_pass_read = students_df[students_df.reading_score > reading_pass].count()["reading_score"]*100/total_students

#% overall passing
pc_pass_both = students_df[(students_df.math_score > maths_pass) & (students_df.reading_score > reading_pass)].count()["math_score"]*100/total_students

#Create summary table
summary_df = pd.DataFrame(
              {"Total Schools": total_schools, "Total Students": total_students, "Total Budget": total_budget, 
               "Average Maths Score": average_maths, "Average Reading Score": average_reading,
              "% passing maths": pc_pass_maths, "% passing reading": pc_pass_read,
              "% passing both": pc_pass_both}, index=[1])

pc_pass_maths

100.0

School Summary
------------------------

In [ ]:
#Group by School Name


#Select columns


#Total students


#Total school budget


#Per student budget


#Average maths score


#Average reading score


#% Passing maths


#% passing reading


#% overall passing

Top performing schools
----------------------------------

In [ ]:
#Sort school summary


#Select top 5

Bottom performing schools
-------------------------------------

In [ ]:
#Sort school summary


#Select bottom 5


Secondary analysis
==================

Maths scores by grade
---------------------

Reading scores by grade
-----------------------

Scores by school spending
-------------------------

In [ ]:
#Define bins


#Select data


#Bin data

Scores by school size
---------------------

In [ ]:
#Define bins


#Select data


#Bin data

Scores by school type
---------------------

In [ ]:
#Define bins


#Select data


#Bin data

Reporting
=========